In [2]:
import numpy as np
from numpy.linalg import cholesky, solve

In [3]:
#define consts 
N = 20
coords_1d = np.linspace(0, 1, N)
X_grid = np.array([(x, y) for x in coords_1d for y in coords_1d])  # (N^2, 2)
n_points = X_grid.shape[0]

g_cut = 0.0
v_val = 1.0
c_d = 0.05
sigma_noise = 0.05


In [6]:
def rbf_kernel(X1, X2, lengthscale=0.2, variance=1.0):
    # squared Euclidean distance
    dists = np.sum(X1**2, axis=1)[:, None] + np.sum(X2**2, axis=1)[None, :] - 2 * X1 @ X2.T
    return variance * np.exp(-0.5 * dists / (lengthscale**2))
K_full = rbf_kernel(X_grid, X_grid, lengthscale=0.25, variance=1.0)

In [18]:
#generate ground truth... 
import numpy as np

# kernel using gaussian 
def gaussian(X1, X2, length=0.25, variance=1.0):
    # squared distances
    X1_sq = np.sum(X1**2, axis=1)[:, None]      # (n1, 1)
    X2_sq = np.sum(X2**2, axis=1)[None, :]      # (1, n2)
    dists = X1_sq + X2_sq - 2 * (X1 @ X2.T)     # (n1, n2)
    K = variance * np.exp(-0.5 * dists / (length**2))
    return K
    #k =[n1, n2] w covariance K_ij = var * exp(-0.5 * || x_i - x_j || ^ 2 / length^2)

#generate ground truth, we sample one true ore from the GP prior over thr grid and return an array of grades
def sample_ground_truth(X_grid, length=0.25, variance=1.0, jitter=1e-6):
    n = X_grid.shape[0]
    K_full = gaussian(X_grid, X_grid, length=length, variance=variance)
    # numerical jitter on the diagonal
    K_full = K_full + jitter * np.eye(n)
    # Cholesky factorization: K = L L^T
    L = np.linalg.cholesky(K_full)
    # sample z ~ N(0, I)
    z = np.random.randn(n)
    # correlated sample: G_true ~ N(0, K)
    G_true = L @ z
    return G_true

G_true = sample_ground_truth(X_grid, length=0.25, variance=1.0)
G_true_grid = G_true.reshape(N, N)  # for plotting later
print("Ground truth field shape:", G_true_grid.shape)


Ground truth field shape: (20, 20)


In [4]:
#Initialize prior over grades 
#before drilling every cell has grade 0 , correlations encoded in covariance matrix 
mu = np.zeros(n_points)
Sigma = K_full

In [8]:
#single drilling observation + gaussian posterior update given the data we observe

#Kalman filter esque, update given one new noisy observation at index i across nxn, should probably maybe ij
def update_posterior(mu, Sigma, i, y_i, sigma2):
    #sigma2 is noise variance, yi is observed grade, mu / sigma are current mean / covariance, i = cell index
    # predictive mean and variance of this observation under the current belief
    y_hat = mu[i]                 # E[y_i | current belief]
    S = Sigma[i, i] + sigma2      # Var[y_i | current belief]
    # how surprising was the observation
    e = y_i - y_hat               # if e ~ 0 no surprise, if big, we adjust more
    g = Sigma[:, i] / S         
    # shift each cell proportional to gain * surprise, updated mean
    mu_new = mu + g * e       
    # updated covariance 
    Sigma_new = Sigma - np.outer(g, g) * S   # (n_points, n_points)

    return mu_new, Sigma_new



In [9]:
#posterior sampling, sample one full ore field from the current posterior, thompson sampling? not yet implemented
def sample_posterior(mu, Sigma, jitter=1e-6):
    n = mu.shape[0]
    # numerical jitter to make Sigma positive definite
    Sigma_j = Sigma + jitter * np.eye(n)
    L = np.linalg.cholesky(Sigma_j)
    z = np.random.randn(n)
    return mu + L @ z


In [15]:
#main 
N = 20
coords_1d = np.linspace(0.0, 1.0, N)
X_grid = np.array([(x, y) for x in coords_1d for y in coords_1d])
n_points = X_grid.shape[0]
K_full = gaussian(X_grid, X_grid, lengthscale=0.25, variance=1.0)

# sample from the hidden body
G_true = sample_ground_truth(X_grid, lengthscale=0.25, variance=1.0)
mu = np.zeros(n_points)  #0 
Sigma = K_full.copy()     #spatial correlation, gaussian normal 

sigma_obs = 0.05     
sigma2_obs = sigma_obs**2
B = 10                    # drill no.

drilled_indices = []

for t in range(B):
    # pick a random cell
    candidates = np.setdiff1d(np.arange(n_points), np.array(drilled_indices, dtype=int))
    i = np.random.choice(candidates)
    drilled_indices.append(i)
    # observe noisy output ore grade at that cell from the hidden true field
    y_i = G_true[i] + sigma_obs * np.random.randn()
    # update posterior belief
    mu, Sigma = update_posterior(mu, Sigma, i, y_i, sigma2_obs)
    print(f"Step {t+1}: drilled cell {i}, observed y_i={y_i:.3f}")

# After B drills, mu and Sigma are your posterior belief over the whole grid
print("Posterior mean shape:", mu.shape)
print("Posterior covariance shape:", Sigma.shape)


Step 1: drilled cell 130, observed y_i=-1.055
Step 2: drilled cell 34, observed y_i=-0.607
Step 3: drilled cell 121, observed y_i=0.564
Step 4: drilled cell 347, observed y_i=1.503
Step 5: drilled cell 2, observed y_i=1.495
Step 6: drilled cell 204, observed y_i=0.954
Step 7: drilled cell 387, observed y_i=1.583
Step 8: drilled cell 294, observed y_i=1.380
Step 9: drilled cell 332, observed y_i=1.846
Step 10: drilled cell 111, observed y_i=-1.088
Posterior mean shape: (400,)
Posterior covariance shape: (400, 400)
